In [1]:
import time, re
import numpy as np
from math import e
from typing import Dict, List, Set
from nptyping import NDArray, Float64
from Bio import Entrez, Medline

In [2]:
with open( 'positive_examples.txt' ) as file :
    posids = set( file.read().split( ',' ) )

In [3]:
with open( 'negative_examples.txt' ) as file :
    negids = set( file.read().split( ',' ) )

In [4]:
Count = int
Class, Document, Word = str, str, str

In [5]:
data : Dict[ Class, Set[ Document ] ] = { 'positive' : set(), 'negative' : set() }

In [6]:
# getting data
for pmid in posids :
    data[ 'positive' ].add(
        Medline.read(
            Entrez.efetch(
                db = 'pubmed',
                id = pmid,
                email = 'chiodini.zachary@epa.gov',
                retmode = 'text',
                rettype = 'medline'
                )
            ).get( 'AB' )
        )
    time.sleep( 1/3 ) # avoid PubMed ban
for pmid in negids :
    data[ 'negative' ].add(
        Medline.read(
            Entrez.efetch(
                db = 'pubmed',
                id = pmid,
                email = 'chiodini.zachary@epa.gov',
                retmode = 'text',
                rettype = 'medline'
                )
            ).get( 'AB' )
        )
    time.sleep( 1/3 ) # avoid PubMed ban

In [42]:
class LogisticRegression :
    '''
    Binomial Logistic Regression for Text Classification
    '''
    def __init__( self ) :
        self.network = np.array([])    # weights and bias
        self.vocabulary = np.array([]) # features
        self.__vecfindall = np.vectorize( 
            lambda voc, doc : int( voc in doc ) 
            )
        self.stop_words = {}
        with open( 'stopset.txt' ) as file :
            self.stop_words = set( file.read().split( ',' ) )
            
    def train( self, 
        data : Dict[ Class, Set[ Document ] ],
        rate : float = 1.0,
        batches : int = 10,
        convergence : float = 0.01,
        ) -> None :
        # get features
        self.vocabulary = np.array( list( 
            self.__removeFirstQ( self.__extract( data ) )
            ) )
        # append bias
        self.vocabulary = np.append( self.vocabulary, '' )
        # generate random weights
        self.network = np.random.uniform( 
            -0.5, 0.5, size = len( self.vocabulary )
            )
        # generate input vectors
        X = np.empty( shape = ( 0, len( self.vocabulary ) ) )
        for label in data :
            for document in data[ label ] :
                X = np.vstack( ( X, self.getInputFrom( document ) ) )
        # get target values
        target = list( data )[ 0 ]
        Y = np.array( [ int( label == target ) for label in data 
                        for document in data[ label ] ] )
        totgrad = np.inf
        while totgrad > convergence :
            totgrad = 0
            for xbatch, ybatch in zip( 
                np.array_split( X, batches ), 
                np.array_split( Y, batches ) 
                ) :
                grad = np.multiply( 
                    np.reshape( 
                        self.output( xbatch ) - ybatch, 
                        newshape = ( len( xbatch ), 1 )
                        ),
                    xbatch
                    ).sum( axis = 0 ) / len( xbatch )
                self.network = self.network - rate*grad
                totgrad += grad.sum() / len( grad )
        return
    
    def test( self, 
        data : Dict[ Class, Set[ Document ] ], 
        boundary : float = 0.5 
    ) -> None :
        pass
            
    def getInputFrom( self, document : str ) -> NDArray[ int ] :
        ''' Generate Input Vector '''
        return self.__findall( self.vocabulary, document )
            
    def output( self, X : NDArray[ int ] ) :
        ''' Logistic Model Output '''
        return self.sigmoid( np.dot( X, self.network ) )
            
    def sigmoid( self, x : float ) -> float :
        return 1 / ( 1 + e**(-x) )
    
    def __findall( self, 
        vocabulary : NDArray[ Word ], 
        document   : str
        ) -> NDArray[ int ] :
        return self.__vecfindall( vocabulary, document )

    def __extract( self,
        data : Dict[ Class, Set[ Document ] ],
        pattern : str = '\\b[a-z]{2,}\\b'
        ) -> Dict[ Word, Count ] :
        '''
        Extract Vocabulary from Dataset
        '''
        vocabulary : Dict[ Word, Count ] = {}
        for label in data :
            for document in data[ label ] :
                for word in re.findall(
                    pattern = pattern,
                    string  = document
                    ) :
                    if word not in self.stop_words :
                        if word in vocabulary :
                            vocabulary[ word ] += 1
                        else :
                            vocabulary[ word ] = 1
        return vocabulary
    
    def __removeFirstQ( self,
        vocabulary : Dict[ Word, Count ]
        ) -> Dict[ Word, Count ] :
        '''
        Remove First Quartile in Vocabulary
        '''
        count = sorted( set( vocabulary.values() ) )
        index = len( count ) / 4
        if index % 1 == 0 :
            limit = count[ int( index ) ]
        else :
            index = int( index ) # truncate
            limit = ( count[ index ] + count[ index + 1 ] ) / 2
        for word, count in vocabulary.copy().items() :
            if count < limit :
                del vocabulary[ word ]
        return vocabulary

In [43]:
model = LogisticRegression()

In [44]:
model.train( data )

In [47]:
model.network

array([ 0.05063314, -0.7412253 ,  0.77752734, -0.0122118 , -0.40372418,
       -0.00409059,  0.81301947,  0.18197832,  0.79984464,  0.43732911,
       -0.41077634,  0.94513105,  0.12216768,  0.05564902,  0.68189066,
        0.39209664, -0.31632403, -0.46583514,  0.27344065, -0.19113349,
        0.06174579,  0.98470539, -0.25166816,  0.87969516,  0.24531371,
        0.81272763, -0.01590201,  0.10830178, -0.52954956, -0.3696967 ,
        0.5619696 ,  0.10280133, -0.11656916, -0.06118743,  0.49972869,
        1.11599386,  0.20016204, -0.04948729, -0.40842085, -0.46140762,
       -0.42006598, -0.02340598,  0.07638705, -0.29508413, -0.03422346,
       -0.04968193,  0.47553087,  0.60268718,  0.37937401, -0.89076507,
       -0.00307398,  0.13066231,  0.42321262,  0.03048048,  0.24700911,
       -0.12092759,  0.06873496, -0.28533924, -0.47837386, -0.09516471,
       -0.15620561,  0.39970991,  0.05045602, -0.20935676,  0.14729556,
        0.30204452, -0.41596891,  0.21015156,  0.74266737,  0.10